# Preprocessing of Fleet_deployment from LLoyd dataset . Combining Ais data with LLoyd

In [ ]:
import pandas as pd

In [21]:
df=pd.read_csv("/home/talal/fyp/predictive model/Lloyds/ship_data/tblShip.csv")

In [23]:
pd.set_option('display.max_columns', None)

In [ ]:
selected_columns = df[['LRIMOShipNo', 'ShipName', 'ShipStatus',"MaritimeMobileServiceIdentityMMSINumber","ShipManagerCompanyCode"]]

In [90]:
import pandas as pd

df2=pd.read_csv("/home/talal/fyp/predictive model/Lloyds/ship_data/tblCompanyFullDetailsAndParentCode.csv",dtype={'OWCODE': str})
df1=pd.read_csv("/home/talal/fyp/predictive model/Lloyds/ship_data/tblShip.csv",dtype={'ShipManagerCompanyCode': str,"MaritimeMobileServiceIdentityMMSINumber":str})
df1 = df1[df1['MaritimeMobileServiceIdentityMMSINumber'].notna()]
df1 = df1[['LRIMOShipNo', 'ShipName', 'ShipStatus',"MaritimeMobileServiceIdentityMMSINumber","ShipManagerCompanyCode"]]
merged_df = pd.merge(df1, df2[['OWCODE', 'ShortCompanyName']], 
                     left_on='ShipManagerCompanyCode', 
                     right_on='OWCODE', 
                     how='left')

df = merged_df

/tmp/ipykernel_6293/4279810212.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df2=pd.read_csv("/home/talal/fyp/predictive model/Lloyds/ship_data/tblCompanyFullDetailsAndParentCode.csv",dtype={'OWCODE': str})


In [92]:
df.to_csv("Ship_company_name.csv")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
import pyspark.sql.functions as F

# Create Spark session
spark = SparkSession.builder \
    .appName("AIS Data Processing - MMSI Matching with CSV") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/ais_training_data.specific_mmsi_data") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/ais_training_data.new_specific_mmsi_data_cname") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.driver.memory", "8g") \
    .config("spark.mongodb.output.bulk.enabled", "true") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Load CSV data into a Spark DataFrame
ship_data_df = spark.read.csv("/home/talal/fyp/predictive model/Ship_company_name.csv", header=True, inferSchema=True)

# Selecting relevant columns from the CSV (MMSI and ShortCompanyName)
ship_data_df = ship_data_df.select("MaritimeMobileServiceIdentityMMSINumber", "ShortCompanyName")

# Read MongoDB data

skip_documents = 1568093  
df = spark.read.format("mongo").option("spark.mongodb.input.skip", skip_documents) \
    .option("spark.mongodb.input.limit", 196504).option("bulkRead", "true").load()

# Join the MongoDB data with the CSV data based on MMSI
df_with_company_name = df.join(
    ship_data_df,
    df["MMSI"] == ship_data_df["MaritimeMobileServiceIdentityMMSINumber"],
    how="left"  # Use left join to keep all documents from MongoDB even if there's no match in the CSV
)

# Optionally, drop the unnecessary columns and rename columns as needed
df_with_company_name = df_with_company_name.drop("MaritimeMobileServiceIdentityMMSINumber") \
    .withColumnRenamed("ShortCompanyName", "CN")

# Create a new collection by saving the updated DataFrame back to MongoDB
df_with_company_name.write.format("mongo").mode("append").option("bulkWrite", "true").save()

# Print completion message
print("MMSI matching with CSV completed, and data saved in new collection.")

# Stop Spark session
spark.stop()


24/12/10 07:49:17 WARN Utils: Your hostname, talal resolves to a loopback address: 127.0.1.1; using 192.168.10.139 instead (on interface wlo1)
24/12/10 07:49:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/talal/.ivy2/cache
The jars for the packages stored in: /home/talal/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c924f53f-3d7d-4e45-be2d-e54887ac7a5a;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/talal/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 141ms :: artifacts dl 5ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   4   |   0   |   0   |   0   ||   4   |   0   |
	---------------------------------------------------------------------
:: retrieving ::

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
import pyspark.sql.functions as F

# Create Spark session
spark = SparkSession.builder \
    .appName("AIS Data Processing - MMSI Matching with CSV") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/ais_training_data.ais_data") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/ais_training_data.specific_mmsi_data") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

# Load CSV data into a Spark DataFrame
ship_data_df = spark.read.csv("ship_data.csv", header=True, inferSchema=True)

# Selecting relevant columns from the CSV (MMSI and ShortCompanyName)
ship_data_df = ship_data_df.select("MaritimeMobileServiceIdentityMMSINumber", "ShortCompanyName")

# Read MongoDB data
df = spark.read.format("mongo").option("bulkRead", "true").load()

# Join the MongoDB data with the CSV data based on MMSI
df_with_company_name = df.join(
    ship_data_df,
    df["MMSI"] == ship_data_df["MaritimeMobileServiceIdentityMMSINumber"],
    how="left"  # Use left join to keep all documents from MongoDB even if there's no match in the CSV
)


df_with_company_name = df_with_company_name.drop("MaritimeMobileServiceIdentityMMSINumber") \
    .withColumnRenamed("ShortCompanyName", "CompanyName")

# Create a new collection by saving the updated DataFrame back to MongoDB
df_with_company_name.write.format("mongo").mode("overwrite").option("bulkWrite", "true").save()

# Print completion message
print("MMSI matching with CSV completed, and data saved in new collection.")

# Stop Spark session
spark.stop()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, lit, desc
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("AIS Data Processing") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/ais_training_data.new_specific_mmsi_data_cname") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/ais_training_data.ais_data_summary") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.mongodb.input.batchSize", "100000") \
    .config("spark.mongodb.output.batchSize", "10000") \
    .config("spark.driver.memory", "8g") \
    .config("spark.mongodb.output.bulk.enabled", "true") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Read data from MongoDB
df = spark.read.format("mongo").option("bulkRead", "true").load()


# Write the sorted result to a new MongoDB collection
df.write.format("mongo") \
    .mode("overwrite") \
    .option("database", "ais_training_data") \
    .option("collection", "ais_data_summary") \
    .save()

print("Data processing completed and results stored in MongoDB.")

# Stop Spark session
spark.stop()

24/12/10 08:19:02 WARN Utils: Your hostname, talal resolves to a loopback address: 127.0.1.1; using 192.168.10.139 instead (on interface wlo1)
24/12/10 08:19:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/talal/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/talal/.ivy2/cache
The jars for the packages stored in: /home/talal/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f5dee3ae-2ae1-4e98-86bc-535f99f2243a;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 139ms :: artifacts dl 5ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   

Data processing completed and results stored in MongoDB.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# Initialize Spark session
spark = SparkSession.builder \
    .appName("AIS Data Processing") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/ais_training_data.ais_data_summary") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/ais_training_data.ais_data_summary_new") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.driver.memory", "8g") \
    .config("spark.mongodb.output.bulk.enabled", "true") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Read data from MongoDB
df_mongo = spark.read.format("mongo").option("bulkRead", "true").load()

# Load only "Port Name" and "ShortCompanyName" from CSV
df_csv = spark.read.option("header", "true").csv("/home/talal/fyp/predictive model/new_port.csv")
df_csv = df_csv.select("Port Name", "ShortCompanyName")

# Rename columns for consistency with MongoDB data
df_csv = df_csv.withColumnRenamed("Port Name", "Destination")

# Perform a left join on the "Destination" field from MongoDB and "Port Name" from CSV
df_joined = df_mongo.join(df_csv, on="Destination", how="left")

# Update the "cn" field with "ShortCompanyName" if "cn" is null
df_final = df_joined.withColumn(
    "CN", 
    when(col("CN").isNull(), col("ShortCompanyName")).otherwise(col("CN"))
)


df_final = df_final.drop("ShortCompanyName")

# Write the final result to MongoDB
df_final.write.format("mongo") \
    .mode("overwrite") \
    .option("database", "ais_training_data") \
    .option("collection", "ais_data_summary_new") \
    .save()

print("Data processing completed and results stored in MongoDB.")

# Stop Spark session
spark.stop()


24/12/10 11:07:24 WARN Utils: Your hostname, talal resolves to a loopback address: 127.0.1.1; using 192.168.10.139 instead (on interface wlo1)
24/12/10 11:07:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/talal/.ivy2/cache
The jars for the packages stored in: /home/talal/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2fc98cda-a88c-4803-96d7-fdb74ed094c7;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/talal/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 116ms :: artifacts dl 5ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   4   |   0   |   0   |   0   ||   4   |   0   |
	---------------------------------------------------------------------
:: retrieving ::

Data processing completed and results stored in MongoDB.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, weekofyear, year, concat, lit, lpad, desc

# Initialize Spark session
spark = SparkSession.builder \
    .appName("AIS Data Summary Processing") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/ais_training_data.ais_data_summary") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/ais_training_data.ais_data_weekly_summary") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

# Read data from MongoDB
df = spark.read.format("mongo").load()

df_with_week = df.withColumn("Year", year(col("Date"))) \
                 .withColumn("Week", weekofyear(col("Date"))) \
                 .withColumn("YearWeek", concat(col("Year"), lit("-"), lpad(col("Week"), 2, "0"))) \

# Group by YearWeek, Ship_Type, Destination, CN and sum the Count
weekly_summary = df_with_week.groupBy("YearWeek", "Ship_Type", "Destination") \
    .agg(count("*").alias("TotalCount")) \
    .orderBy(desc("YearWeek"), desc("TotalCount"))

# Show the results
weekly_summary.show(n=20, truncate=False)

# Write the results to a new MongoDB collection
weekly_summary.write.format("mongo") \
    .mode("overwrite") \
    .option("database", "ais_training_data") \
    .option("collection", "ais_data_weekly_summary") \
    .save()

print("Weekly summary processed and saved to MongoDB.")

# Stop Spark session
spark.stop()


+--------+---------+-------------------+------------------------------+----------+
|YearWeek|Ship_Type|Destination        |CN                            |TotalCount|
+--------+---------+-------------------+------------------------------+----------+
|2023-52 |Cargo    |Esbjerg            |FOB SWATH AS                  |1         |
|2023-52 |Sailing  |Bandholm           |NULL                          |1         |
|2023-52 |Pilot    |Fredericia         |NULL                          |1         |
|2023-52 |Pilot    |Skagen             |NULL                          |1         |
|2023-52 |Military |Fredericia         |NULL                          |1         |
|2023-52 |Tanker   |Asnaesvaerkets Havn|ERIK THUN AB                  |1         |
|2023-52 |Undefined|Hvide Sande        |HOY P                         |1         |
|2023-52 |Tug      |Ronne              |HEEREMA MARINE CONTRACTORS-NTH|1         |
|2023-52 |Other    |Koge               |NULL                          |1         |
|202

Weekly summary processed and saved to MongoDB.


In [14]:
import pandas as pd

# Load the two CSV files into pandas DataFrames
companies_df = pd.read_csv('/home/talal/fyp/predictive model/Lloyds/ship_data/tblCompanyFullDetailsAndParentCode.csv')
ports_df = pd.read_csv('/home/talal/fyp/predictive model/ports_output.csv')

# Convert the country names to lowercase for case-insensitive matching
companies_df['CountryName'] = companies_df['CountryName'].str.lower()
ports_df['Country'] = ports_df['Country'].str.lower()

# Merge the two DataFrames on the country name
merged_df = pd.merge(companies_df, ports_df, left_on='CountryName', right_on='Country', how='left')

final_df = merged_df[['ShortCompanyName', 'CountryName', 'Port Name']].dropna(subset=['Port Name'])


# Save the result to a new CSV file
#final_df.to_csv('merged_companies_with_ports.csv', index=False)

/tmp/ipykernel_237211/732292704.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  companies_df = pd.read_csv('/home/talal/fyp/predictive model/Lloyds/ship_data/tblCompanyFullDetailsAndParentCode.csv')


In [16]:
final_df.to_csv("new_port.csv")

In [6]:


from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, weekofyear, year, concat, lit, lpad, desc, datediff

# Initialize Spark session
spark = SparkSession.builder \
    .appName("AIS Data Summary Processing") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/ais_training_data.ais_data_summary_new") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/ais_training_data.ais_data_relative_week_summary_simple") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

# Read data from MongoDB
df = spark.read.format("mongo").load()

# Define the fixed first date (baseline date)
first_date = "2023-01-01"

# Add columns for Year, Week, and the relative week number
df_with_week = df.withColumn("Year", year(col("Date"))) \
                 .withColumn("Week", weekofyear(col("Date"))) \
                 .withColumn("RelativeWeek", (datediff(col("Date"), lit(first_date)) / 7).cast("int") + 1) \
                 .withColumn("YearWeek", concat(col("Year"), lit("-"), lpad(col("RelativeWeek"), 2, "0")))

# Group by the relative week, Ship_Type, Destination, CN and sum the Count
relative_week_summary = df_with_week.groupBy("YearWeek", "Ship_Type", "Destination") \
    .agg(count("*").alias("TotalCount")) \
    .orderBy(desc("YearWeek"), desc("TotalCount"))

# Show the results
relative_week_summary.show(n=20, truncate=False)

# Write the results to a new MongoDB collection
relative_week_summary.write.format("mongo") \
    .mode("overwrite") \
    .option("database", "ais_training_data") \
    .option("collection", "ais_data_relative_week_summary_simple") \
    .save()

print("Relative week summary processed and saved to MongoDB.")

# Stop Spark session
spark.stop()



+--------+---------+-----------+----------+
|YearWeek|Ship_Type|Destination|TotalCount|
+--------+---------+-----------+----------+
|2023-43 |Undefined|Rostock    |26        |
|2023-43 |Undefined|Esbjerg    |24        |
|2023-43 |Undefined|Sassnitz   |24        |
|2023-43 |Undefined|Skagen     |22        |
|2023-43 |Undefined|Gotthenburg|21        |
|2023-43 |Undefined|Rendsburg  |20        |
|2023-43 |Undefined|Holtenau   |19        |
|2023-43 |Undefined|Brunsbuttel|18        |
|2023-43 |Undefined|Visby      |17        |
|2023-43 |Undefined|Rodbyhavn  |17        |
|2023-43 |Undefined|Cuxhaven   |16        |
|2023-43 |Undefined|Busum      |14        |
|2023-43 |Undefined|Hvide Sande|13        |
|2023-43 |Cargo    |Rostock    |13        |
|2023-43 |Undefined|Grenaa     |13        |
|2023-43 |Undefined|Kerteminde |11        |
|2023-43 |Cargo    |Esbjerg    |11        |
|2023-43 |Undefined|Butzfleth  |10        |
|2023-43 |Undefined|Hirtshals  |10        |
|2023-43 |Undefined|Gluckstadt |

Relative week summary processed and saved to MongoDB.
